<a href="https://colab.research.google.com/github/anmaxwell/UniNotebooks/blob/master/2AssesB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install scattertext

     |████████████████████████████████| 6.8MB 9.3MB/s 


In [0]:
!pip install "git+https://github.com/facebookresearch/fastText.git"

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-ru0og55q
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-ru0og55q
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2871262 sha256=af7db4f0e64592d32ba9c037c0305cdd601cdfe2c862779433e3bcee68281d8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ycnfdviu/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext


Install all necessary packages

In [0]:
import fasttext.util
import numpy as np
import pandas as pd
import re
import scattertext as st
import spacy

from keras import layers
from keras.models import Sequential
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from scattertext import CorpusFromPandas, produce_scattertext_explorer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Read in data and look at first item

In [0]:
df = pd.read_csv('agr_en_train.csv', names=['unique_id','text','aggression-level'], sep=',')
print(df.iloc[0])

unique_id                                 facebook_corpus_msr_1723796
text                Well said sonu..you have courage to stand agai...
aggression-level                                                  OAG
Name: 0, dtype: object


Check for missing values

In [0]:
df.isna().values.any()

False

Count the occurrences per Aggression Level

In [0]:
df['aggression-level'].value_counts() 

NAG    5051
CAG    4240
OAG    2708
Name: aggression-level, dtype: int64

Identify the top occurring words per Aggression Level

In [0]:
nlp = spacy.load('en')
df['parsed'] = df.text.apply(nlp)
data = st.CorpusFromParsedDocuments(df, category_col='aggression-level', 
                                      parsed_col='parsed').build().remove_terms(nlp.Defaults.stop_words, ignore_absences=True)

freq_df = data.get_term_freq_df()
oag_tw = freq_df.sort_values(by=['OAG freq'], ascending=False)
oag_tw = oag_tw.drop(oag_tw.columns[[1,2]], axis=1)
nag_tw = freq_df.sort_values(by=['NAG freq'], ascending=False)
nag_tw = nag_tw.drop(nag_tw.columns[[0,2]], axis=1)
cag_tw = freq_df.sort_values(by=['CAG freq'], ascending=False)
cag_tw = cag_tw.drop(cag_tw.columns[[0,1]], axis=1)

print(oag_tw.head())
print(nag_tw.head())
print(cag_tw.head())

        OAG freq
term            
u            420
india        396
people       372
like         317
indian       279
        NAG freq
term            
india        616
people       357
good         355
indian       350
like         315
        CAG freq
term            
people       554
india        493
u            466
bjp          357
like         337


Replace the text labels with values and save as a vector

In [0]:
df['aggression-level'] = df['aggression-level'].replace({ 'OAG' : 0, 'NAG' : 1, 'CAG' : 2 }) 
labels = df['aggression-level'].values
labels = to_categorical(labels, num_classes = 3)

Load the fasttext model

In [0]:
fasttext.util.download_model('en', if_exists='ignore') 
ft = fasttext.load_model('cc.en.300.bin')

Buffered data was truncated after reaching the output size limit.

Set the parameters

In [0]:
review_length = 50
data_count = len(df)
dims = ft.get_dimension()

Sub-function to clean the text, remove all non-ASCII characters and split to individual words

In [0]:
def clean_up(text):
 
  text = text.replace('&', ' and ')
  text = text.replace('@', ' at ')
  text = re.sub(r'[^\x41-\x7f]',r' ',text)
  text = text.lower().split()

  return text

Sub-function to create the vectors

In [0]:
def text_to_vector(text):

    text = normalize(text)
    words = text.split()
    window = words[-review_length:]
    
    x = np.zeros((review_length, dims))

    for i, word in enumerate(window):
        x[i, :] = ft.get_word_vector(word).astype('float32')

    return x


Function to take the text from the review column, clean it, turn it to individual words then convert to vectors

In [0]:

def df_to_data(df):
    """
    Convert a given dataframe to a dataset of inputs for the NN.
    """
    x = np.zeros((len(df), review_length, dims), dtype='float32')

    for i, comment in enumerate(df['text'].values):
        x[i, :] = text_to_vector(comment)

    return x

In [0]:
embedding = df_to_data(df)

In [0]:
embedding.shape

(11999, 50, 300)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(embedding, labels, test_size=0.20)

In [0]:
from keras.models import Sequential
from keras import layers

In [0]:
def cnn_text_classifier():

    model = Sequential()
    model.add(layers.Conv1D(128, 5, activation='relu', input_shape=(50, 300)))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(3, activation='sigmoid'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [0]:
model = cnn_text_classifier()
training = model.fit(X_train, y_train, epochs=10, verbose=False, validation_data=(X_test, y_test), batch_size=10)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 46, 128)           192128    
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 128)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 33        
Total params: 193,451
Trainable params: 193,451
Non-trainable params: 0
_________________________________________________________________


In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9535
Testing Accuracy:  0.5167
